<a href="https://colab.research.google.com/github/Ishaan20072612/Rag-Architecture-md-Processing/blob/main/Rag_md_Reader_and_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nbstripout

In [ ]:
pip install -qU langchain_community

***Step 1: Downloading The Required Libraries***

In [ ]:
import os
from langchain.document_loaders import TextLoader
from transformers import pipeline
from langchain_text_splitters import MarkdownHeaderTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

***Step 2: Loading The File***

In [ ]:
loader = TextLoader("/content/tennis_details.md") # Copied the file path
text_doc = loader.load() # Loaded the required file
print(text_doc[0].page_content)

# Tennis

## Introduction
Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.

## Basic Rules
- A match can be played as best of three or five sets.
- Each set consists of games, and each game consists of points.
- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.
- A player must win a game by at least **two points**.
- The ball must land within the designated court boundaries.

## Scoring System
```plaintext
0 points  -> Love
1 point   -> 15
2 points  -> 30
3 points  -> 40
4 points  -> Game (if leading by 2)
Deuce     -> 40-40 (must win two consecutive points to win the game)
Advantage -> If a player wins a point at deuce, they gain the advantage
```

## Famous Tournaments
- **Grand Slam Events**:
  - Australian Open
  - French Open
  - Wimbledon
  - US Open

## Equipment
- **Racket**: Used to hit the ball.
- **Tennis Ball**: Yello

***Step 3: Splitting The Text***

In [ ]:
split_cond = [ ( "##" , "Title" ) ]
splitter = MarkdownHeaderTextSplitter(split_cond) #Splitting the text into the chunks
splitti = splitter.split_text(text_doc[0].page_content)
print(splitti)

[Document(metadata={}, page_content='# Tennis'), Document(metadata={'Title': 'Introduction'}, page_content="Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court."), Document(metadata={'Title': 'Basic Rules'}, page_content='- A match can be played as best of three or five sets.\n- Each set consists of games, and each game consists of points.\n- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.\n- A player must win a game by at least **two points**.\n- The ball must land within the designated court boundaries.'), Document(metadata={'Title': 'Scoring System'}, page_content='```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n```'

In [ ]:
text_chunks = []
for i in splitti:
  text_chunks.append(i.page_content)
print(text_chunks)
len(text_chunks)

['# Tennis', "Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.", '- A match can be played as best of three or five sets.\n- Each set consists of games, and each game consists of points.\n- Points are scored as **0 (Love), 15, 30, 40**, and then **game**.\n- A player must win a game by at least **two points**.\n- The ball must land within the designated court boundaries.', '```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n```', '- **Grand Slam Events**:\n- Australian Open\n- French Open\n- Wimbledon\n- US Open', '- **Racket**: Used to hit the ball.\n- **Tennis Ball**: Yellow-green in color, designed for optimal bounce.\n- **Court**: Ca

7

***Step 4: Creating Embeddings***

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
def embed_chunk(chunk):
  return embedding_model.encode([chunk] , normalize_embeddings=True)
sample_embeddings = embed_chunk(text_chunks)
print(sample_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[ 2.45774128e-02  1.48012033e-02  5.18463142e-02 -7.59667307e-02
  -9.86059383e-02  1.63238943e-02  8.79042596e-02  6.02516904e-02
   8.84635821e-02  1.29884779e-01 -5.67051843e-02  4.10786681e-02
  -2.85654585e-03  2.36859098e-02  1.62319243e-02 -3.24991830e-02
   2.05462649e-02 -2.89406832e-02  5.37383221e-02  1.23083852e-02
   3.42428908e-02 -4.62832972e-02  3.65828909e-02 -9.38248560e-02
  -1.99453942e-02  3.73774586e-04 -7.28232265e-02  6.46518096e-02
  -3.76451164e-02  4.21944186e-02 -5.50501198e-02  3.23233567e-02
  -2.31778119e-02  4.07124087e-02 -1.92247316e-01 -3.21713723e-02
  -4.56059910e-02  5.64203039e-02 -1.90198049e-02  4.14559804e-02
   4.61168624e-02 -5.83031811e-02  1.99592263e-02  6.55939728e-02
  -4.69155377e-04  1.11691475e-01 -4.80918698e-02  1.13276400e-01
   3.87044288e-02 -4.23419066e-02 -7.18034506e-02  4.09240760e-02
   4.01694626e-02 -4.08083061e-03  9.91938785e-02 -4.47076571e-04
   3.92496400e-02  1.89655870e-02  1.94422994e-02  1.09847514e-02
   7.41450

***Step 5: Storing into Vector DataBase***

In [ ]:
!pip install -qU chromadb
vector_db = Chroma.from_texts( text_chunks , HuggingFaceEmbeddings( model_name = "all-MiniLM-L6-v2" ) , persist_directory = "/tmp/chromadb/" )
vector_db._collection.get(include=["embeddings", "documents"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 477.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s et

/tmp/ipython-input-1932018546.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_db = Chroma.from_texts( text_chunks , HuggingFaceEmbeddings( model_name = "all-MiniLM-L6-v2" ) , persist_directory = "/tmp/chromadb/" )
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'ids': ['89499fe1-bbcb-4a47-9a68-b2fe9115fe4b',
  'd1a3fdcf-eae8-491d-aeec-53a65bc02a11',
  '3132e92f-5745-4506-91ea-bc773a80c21c',
  'b00eb7b2-963a-4b80-a1d5-f672a4238ef0',
  '31ea298e-7457-49f6-b2cc-e285f18dde0e',
  '0ca275b1-ed88-437c-8d4c-374a84eee85e',
  'c3f3c282-670c-47db-a985-3ac3c7489dc2'],
 'embeddings': array([[ 0.0227568 ,  0.05737348,  0.06708645, ..., -0.09128203,
          0.03132669,  0.02229537],
        [ 0.04312836,  0.013731  ,  0.04093744, ..., -0.01512668,
         -0.00087324,  0.03250713],
        [ 0.03571488,  0.02733695, -0.01912353, ...,  0.03562167,
         -0.01786362,  0.02190564],
        ...,
        [ 0.01810161,  0.02553317,  0.02062308, ..., -0.07975577,
         -0.04529488,  0.0281473 ],
        [ 0.05494464,  0.04179734,  0.03473552, ..., -0.04557597,
          0.05043149,  0.04526827],
        [ 0.02283715,  0.03019896,  0.07116921, ..., -0.01202571,
         -0.01404738,  0.03577407]]),
 'documents': ['# Tennis',
  "Tennis is a popular sport p

***Step 6: Logging to HuggingFace Hub***

In [ ]:
from transformers import pipeline
from huggingface_hub import login
login()

***Step 7: Using the pipeline to generate the text***

In [ ]:
pipe = pipeline("text-generation" , model = "meta-llama/Llama-3.2-1B")
print(pipe)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cpu


***Step 8: Setting up a Retrieval-based QA Chain***

In [ ]:
def retrieve_and_generate(query,threshold=1):
  search_results = vector_db.similarity_search_with_score(query,k=1)
  if not search_results or search_results[0][1] > threshold:
    return "I don't know the answer"
  retrieve_context = search_results[0][0].page_content
  similarity_score = search_results[0][1]
  print(f"Similarity Score: {similarity_score}")
  print(f"Retrieved Context: {retrieve_context}")
  prompt = f"Answer the question using the given context \n Context: {retrieve_context} \n Question: {query} \n Answer:"
  response = pipe(prompt , max_new_tokens = 100)
  print(response)
  return response[0]["generated_text"]

***Step 9: Asking a question and retrieving the answer***

In [ ]:
ques = "What is tennis?"
response = retrieve_and_generate(ques)
print(response)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Similarity Score: 0.29123157262802124
Retrieved Context: Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court.
[{'generated_text': "Answer the question using the given context \n Context: Tennis is a popular sport played between two players (singles) or two teams of two players each (doubles). The game involves using a racket to hit a ball over a net into the opponent's court. \n Question: What is tennis? \n Answer: Tennis is a sport that involves hitting a ball back and forth between two players over a net. The object is to hit the ball so that it lands in the opponent's court. The winner is the first player to score 4 points, with a match lasting 3 sets of 6 points each. The game is played on a hard court with a tennis court net at the center of the court. The players use a racket to hit the ball back and forth, and the ball is not allowed"}]
Answe

In [ ]:
ques = "What is gated models?"
response = retrieve_and_generate(ques)
print(response)

I don't know the answer


In [ ]:
ques = "In a deuce situation, explain step-by-step how a player can lose the game even after gaining the advantage."
response = retrieve_and_generate(ques)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Similarity Score: 0.8098893761634827
Retrieved Context: ```plaintext
0 points  -> Love
1 point   -> 15
2 points  -> 30
3 points  -> 40
4 points  -> Game (if leading by 2)
Deuce     -> 40-40 (must win two consecutive points to win the game)
Advantage -> If a player wins a point at deuce, they gain the advantage
```
[{'generated_text': "Answer the question using the given context \n Context: ```plaintext\n0 points  -> Love\n1 point   -> 15\n2 points  -> 30\n3 points  -> 40\n4 points  -> Game (if leading by 2)\nDeuce     -> 40-40 (must win two consecutive points to win the game)\nAdvantage -> If a player wins a point at deuce, they gain the advantage\n``` \n Question: In a deuce situation, explain step-by-step how a player can lose the game even after gaining the advantage. \n Answer: Let's say we are at deuce, 40-40, and we have the advantage. The player will lose the game if they win the next point. The player would have to win two consecutive points to win the game. Let's say they win 